# Textual Manipulation

When we are looking at our test table we can see a glaring problem with the data as presented:

In [ ]:
select *
from JupyterDatabase.sales.basicSales;

We cannot readily use the information which our application has passed to the database in terms of our employee who made the sales.

Luckily there are several SQL functions available to us in order to manipulate text to suit our needs.

#### REPLACE

As you can imagine, this function serves an obvious purpose of allowing us to replace something in a string with something else.

It works in the following way:

<code>REPLACE(string, whatToLookFor, whatToReplaceWith)</code>

To use this in a quick demo let's take the phrase 'This is txt' which I don't like and therefore will correct to say 'This is text'.

In order to achieve this I need to look in the text string and ask SQL to replace the letter 'x' with 'ex':

In [ ]:
select replace('This is txt', 'x', 'ex');

As you can see this worked perfectly.

I also wanted to highlight that this doesn't just have to look for characters but it can replace whole words or phrases as well, therefore we will perform the same replacement but with the word 'txt' instead:

In [ ]:
select replace('This is txt', 'txt', 'text');

This provides us with the same result, as we would expect.

#### Length of Data

Although not strictly manipulation, this can play a part in other manipulation techniques and therefore needs to be mentioned.

If we wish to know how long a string is within our table we can utilise two functions; <code>LEN()</code> and <code>DATALENGTH()</code>

Although appearing to be the same they have slightly different functions and we will explore that now:

In [ ]:
select len('This string has 3 spaces at the end   '),
        datalength('This string has 3 spaces at the end   ');

As you can see from the results of executing the above, the differences between the results is 3.  This shows us that <code>len()</code> returns the trimmed length of the string, whereas <code>datalength()</code> will return the complete length including any white space.

#### TRIM

The term 'trimming a string' refers to the removal of white space from the front and end of a string.

* Left Trim will remove any white spaces from the start of a string
* Right Trim will remove any trailing white spaces
* Trim will remove white space from both the start and end of a string

This is all well and good with one exception, that being that although LTRIM and RTRIM (left and right trim respectively) have been supported in SQL Server for a long time, TRIM as a function only appeared in SQL Server 2017 and therefore any version older than that must rely on a combination of LTRIM and RTRIM to obtain the same result.

In [ ]:
select ltrim('     Removes left spaces only     '),
		rtrim('     Removes right spaces only     '),
		trim('     Removes left and right spaces     '), -- SQL Server 2017 and above
		ltrim(rtrim('     Removes left and right spaces     ')); -- SQL Server 2016 and below

As you can see above, it's cleaner and neater to use the newer syntax of TRIM when wanting to clear white space from both the start and end of a string, but it is still perfectly possible in older versions, it just looks a little more messy.

#### LEFT and RIGHT

Now we can start to control our trimming a little more, taking X characters from the left or the right of a string to create a new one.

These both work in the same way:

`LEFT(string, numCharacters) / RIGHT(string, numCharacters)`

For example, using the phrase "two words" we might take the left 3 characters, leaving us with "two" or the right 5 characters, leaving us with "words".

In [ ]:
select left('two words', 3),
        right('two words', 5);

When using the LEFT and RIGHT functions there is one important consideration which is that, unlike other programming languages, the number of characters that we are looking for has no limit.

For example, the following statement is perfectly valid:

In [ ]:
select left('This string', 1000);

However, in most other languages (such as C#), this would throw an error by claiming that there is no character at position 1000.  But in SQL this is perfectly valid and throws no error.

Anyway, if you remember I mentioned that LEN could be useful in this section? Well, we can now use this in combination with RIGHT in order to answer one of our criteria:

-   Year of order
-   Day of Week order was taken
-   **Employee Name**
-   Customer's Full Name
-   Total Order Value (Sale Amount + Tax)

_Note - there are other ways to do this and we will encounter some more shortly, but for the moment let's look at this method._

If we look at the field in question, we can see that all records have the same pattern:

**Order taken by:** EmployeeName

The phrase 'Order taken by: ' is 16 characters in length.  Using simple maths we can determine that the length of employee name is the length of the entire phrase (LEN of the column) minus 16 (our repeating and consistent phrase).  Hence we can think of that as:

Right(column, len(column) - 16)

Let's try that:

In [ ]:
select right(infoCol, len(infoCol)-16)
from JupyterDatabase.sales.basicSales;

That worked exactly as we had hoped and solves our problem nicely.  As with all programming languages though, there are more than methods available and we shall consider those as this one is very static (if we were to suddenly change the message attached to the record or if the message attached became variable in length then this would no longer work, therefore we'll look for something a little more dynamic).

#### SUBSTRING

This is a common function throughout most languages and the implementation in T-SQL is no different, following the expected usage:

<code>substring(string, startPosition, lengthFromStart)</code>

Let's show this with a demonstration.

Imagine we have the string 'One, Two, Three' and we wish to return 'two' as our result.

In order to do this we would need to start at character position 6 (ONE is 3, the comma is 4, the space is 5, therefore we start at position 6 which is the T in Two), and we would then need to read 3 characters (1=T, 2=w, 3=o).

Putting this all together gives us:

In [ ]:
select substring('One, Two, Three', 6, 3);

So how can this help us in our quest to write our query?  Well, we know that our message 'Order taken by: ' is 16 characters long and therefore we need to look in our column, start at the 17th character, and then go to the end of the string (utilising LEN means that we will always pass in a number which equates to the end or beyond of our string):

In [ ]:
select substring(infoCol, 17, len(infoCol))
from JupyterDatabase.sales.basicSales;

This works perfectly.  However, what if the message was to change? We could find that our code stops working entirely as the new message received from the application is no longer 16 characters.

Therefore SUBSTRING is actually an integral part to finding an answer to our query, but it's incomplete at the moment, we need to combine it with another function.

#### CHARINDEX

Standing for 'character index' this function does exactly as the name implies by finding the position (index) of a character within a given string:

`CHARINDEX(characterToFind, string, startingPosition)`

To provide an example we will use the string 'random,character'.

In order to locate the position the comma symbol lies within the string we need to use the function as stated, looking for the comma and starting at position 1 in the string:

In [ ]:
select charindex(',', 'random,character', 1);

As the above shows, the function located the comma at position 7, which we know to be correct (random having 6 letters and the comma being the next character in the string).

Note that we can also use this to look for the starting character of words (or portion of words) within strings:

In [ ]:
select charindex('char', 'random,character', 1);

Additionally, we can use it to find the second instance of a character (or word or partial word) in a string.

We can do this by altering the starting position of the search.  Looking at 'random,character' again we can look for the position of the second 'r' within the string.  We know that our string starts with an r and therefore we don't want to return that, so we can tell charindex to start at position 2:

In [ ]:
select charindex('r', 'random,character', 2);

That does the trick nicely.

Finally, another use for this versatile function is to tell us if the searched character exists at all.  We'll now look for the letter 'x' in our phrase:

In [ ]:
select charindex('x', 'random,character', 1);

As you can see, instead of giving an error the function returns a zero to let us know that nothing was found.

##### Combining SUBSTRING and CHARINDEX

As we have seen in previous demos it can be more powerful to combine operators in order to provide a thorough solution to a problem and this is no different.

In order to fulfil our criteria of "Employee Name" for our query but in a truly dynamic way, we can utilise CHARINDEX and SUBSTRING to come up with our answer.  Let's break this down:

Firstly, we can see from our pattern that we have a colon and space prior to our employee name.  We can presume that if the message was to change (for example from 'Order taken by: ' to 'Cashier Reference: ') then the 'colon space' combination would remain.  Therefore let's use our CHARINDEX function to locate this in our string (we will use both messages to test this):

In [ ]:
select charindex(': ', 'Order taken by: John Smith', 1),
        charindex(': ', 'Cashier Reference: John Smith', 1);

This works perfectly, giving us 15 and 18 respectively, which we can verify by counting manually.

So now we know this, how can we use it to help us? Well, SUBSTRING could be our friend. We know the starting position of the ': ' in the message and therefore we know that our employee name starts from the position of the colon plus 2 (if colon is position 10, the space is position 11, the name starts at position 12.  Therefore 12 - 10 = 2). This means we can use SUBSTRING as follows:

In [ ]:
select substring('Order taken by: John Smith',
					charindex(': ', 'Order taken by: John Smith', 1) + 2,
						len('Order taken by: John Smith')),
		substring('Cashier Reference: John Smith',
					charindex(': ', 'Cashier Reference: John Smith', 1) + 2,
						len('Cashier Reference: John Smith'));

We can see from the above that we have now obtained the correct answer irrespective of the message in front of the employee name (presuming the message always ends with ': '), and therefore we have a much more dynamic solution to our problem that will survive the test of time and changes in the application.

So let's put this into action with our table and see what happens:

In [ ]:
select infoCol, substring(infoCol, charindex(': ', infoCol, 1) + 2, len(infoCol))
from JupyterDatabase.sales.basicSales;

We now have our answer which we can save for our final query.